In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import random
from joblib import load

random.seed(42)

In [2]:
image_loc = ['HMB_1','HM_2','HM_4','HMB_5','HMB_6']

In [ ]:
def attach_image(dataframe, images_location):
    dfs = []
    for frame, image_dir in zip(dataframes, images_location):

        # Attaching image files to the inputs
        image_base_dir = f'./Data/ROSBAG/Ch2_002/images/{image}'
            
        # Define camera folders
        camera_folders = {
                "left": os.path.join(image_base_dir, "left"),
                "center": os.path.join(image_base_dir, "center"),
                "right": os.path.join(image_base_dir, "right"),
            }
            
        # Create new columns for each camera
        for camera, folder in camera_folders.items():
            column_name = f'{camera}_image_path'
            image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
                
            # Add the image paths to the DataFrame
            frame[column_name] = pd.Series(image_files)

        # Drop any inputs that do not have images
    frame = frame.dropna()

    dfs.append(frame)
        
    updated_dataframe = pd.concat(dfs, ignore_index=True)
    
    return updated_dataframe
imu_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/imu-data.csv')
imu_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/imu-data.csv')
imu_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/imu-data.csv')
imu_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/imu-data.csv')
imu_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/imu-data.csv')
imu_dfs = [imu_hmb_1,imu_hmb_2,imu_hmb_4,imu_hmb_5,imu_hmb_6]
imu_all = attach_image(imu_dfs,image_loc)

In [ ]:
def clean_imu_csv(df,filename):
    df = df.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs', 'header.frame_id'])
    df.to_csv(f'./Data/cleaned_data/{filename}.csv', index = False)
    return df
imu_all = clean_imu_csv(imu_all,'imu_all')

In [ ]:
def prepare_data_with_labels(df, image_columns, label_columns, target_size=(224, 224)):
    """
    Prepares X (images) and y (labels) for training a CNN.

    Parameters:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        image_columns (list): List of column names with image paths (e.g., ['left_image_path', 'center_image_path', 'right_image_path']).
        label_columns (list): List of column names for labels (e.g., ['steering_angle', 'throttle_cp']).
        target_size (tuple): Target size for resizing images (width, height).

    Returns:
        np.ndarray: Array of image data (X).
        np.ndarray: Array of labels (y).
    """
    X_images = []
    y_labels = []

    df = df.dropna()
    
    for _, row in df.iterrows():
        row_images = []
        
        # Process images for the current row
        for col in image_columns:
            image_path = row[col]

            print(f"Looking for image: {image_path}")
            
            if os.path.exists(image_path):
                print(f'Found image path {image_path}')# Ensure file exists
                try:
                    img = Image.open(image_path).resize(target_size)  # Open and resize
                    img_array = np.array(img) / 255.0  # Normalize
                    row_images.append(img_array)
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")
            else:
                print(f"Image not found: {image_path}")
        
        # Check if all images for the row are processed
        if len(row_images) == len(image_columns):
            # Combine images if using multiple cameras
            combined_images = np.concatenate(row_images, axis=-1)  # Shape: (height, width, 3 * number of cameras)
            X_images.append(combined_images)
            
            # Collect the corresponding labels
            y_labels.append(row[label_columns].values)
            
    df.set_index('Time', inplace = True)
    
    # Convert to NumPy arrays
    X = np.array(X_images, dtype=np.float32)
    y = np.array(y_labels, dtype=np.float32)
    
    return X, y
X, y = prepare_data_with_labels(steer,['left_image_path', 'center_image_path', 'right_image_path'],imu_all.columns[1:37])

In [ ]:
dump(X_imu_all, "./Data/saved_variables/X_imu_all.pkl")
dump(y_imu_all, "./Data/saved_variables/y_imu_all.pkl")

In [ ]:
def create_imu_model(input_shape):
    model = Sequential()
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4)

    # Input layer
    model.add(Input(shape=input_shape))

    # First convolutional block
    model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Second convolutional block
    model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Third convolutional block
    model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Fourth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Fifth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Sixth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))  
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(units=37, activation='linear'))
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    history = model.fit(X_train,y_train,
                       validation_data = (X_test,y_test),
                       epochs = 50,
                        callbacks = early_stopping,
                       batch_size = 128)

    return model, history
    
input_shape = (224,224,9)
model, history = create_imu_model(input_shape)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
model.save('./Data/full_models/imu_all.keras')

In [ ]:
def attach_image(dataframe, images_location):
    dfs = []
    for frame, image_dir in zip(dataframes, images_location):

        # Attaching image files to the inputs
        image_base_dir = f'./Data/ROSBAG/Ch2_002/images/{image}'
            
        # Define camera folders
        camera_folders = {
                "left": os.path.join(image_base_dir, "left"),
                "center": os.path.join(image_base_dir, "center"),
                "right": os.path.join(image_base_dir, "right"),
            }
            
        # Create new columns for each camera
        for camera, folder in camera_folders.items():
            column_name = f'{camera}_image_path'
            image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
                
            # Add the image paths to the DataFrame
            frame[column_name] = pd.Series(image_files)

        # Drop any inputs that do not have images
    frame = frame.dropna()

    dfs.append(frame)
        
    updated_dataframe = pd.concat(dfs, ignore_index=True)
    
    return updated_dataframe
brake_hmb_1 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-brake_info_report.csv')
brake_hmb_2 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_2/vehicle-brake_info_report.csv')
brake_hmb_4 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_4/vehicle-brake_info_report.csv')
brake_hmb_5 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_5/vehicle-brake_info_report.csv')
brake_hmb_6 = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_6/vehicle-brake_info_report.csv')
brake_dfs = [brake_hmb_1,brake_hmb_2,brake_hmb_4,brake_hmb_5,brake_hmb_6]
brake_all = attach_image(brake_dfs,image_loc)

In [ ]:
def clean_brake_csv(df,filename):
    df['stationary'] = df['stationary'].astype(int) 
    df = df[['Time','brake_torque_actual','wheel_torque_actual','accel_over_ground','stationary']]
    df.to_csv(f'./Data/cleaned_data/{filename}.csv',index = False)
    return df
brake_all = clean_brake_csv(brake_all,'brake_all')

In [ ]:
X, y = prepare_data_with_labels(brake_all,['left_image_path', 'center_image_path', 'right_image_path'],['brake_torque_actual','wheel_torque_actual','accel_over_ground','stationary'])

In [ ]:
def create_brake_model(input_shape):
    model = Sequential()
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4)

    # Input layer
    model.add(Input(shape=input_shape))

    # First convolutional block
    model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Second convolutional block
    model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Third convolutional block
    model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Fourth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Fifth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Sixth convolutional block
    model.add(Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())

    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(0.5))  
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(units=3, activation='linear'))
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    history = model.fit(X_train,y_train,
                       validation_data = (X_test,y_test),
                       epochs = 50,
                        callbacks = early_stopping,
                       batch_size = 128)

    return model, history

input_shape = (224,224,9)
model, history = create_brake_model(input_shape)